In [2]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Cargamos el modelo y el procesador
model_id = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

# TODO: AÑADIR VARIABLE DE ENTORNO CON EL TOKEN DE HUGGING FACE

c:\Users\ALVARO\Desktop\LatencyZero\notebooks\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 398/398 [00:02<00:00, 147.55it/s, Materializing param=visual_projection.weight]                                
CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
text_model.embeddings.position_ids   | UNEXPECTED |  | 
vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [3]:
COMPONENT_LABELS = [
    "a photo of a computer motherboard circuit board with slots",
    "a photo of a GPU graphics card with cooling fans",
    "a photo of a CPU processor chip",
    "a photo of a hard drive or SSD storage device",
    "a photo of RAM memory stick modules",
    "a photo of a PC case tower chassis",
    "a photo of a power supply unit PSU with cables",
    "a photo of a liquid cooling system with radiator",
    "a photo of a computer case fan",
    "a photo of a CPU cooler heatsink with fan",
    "a photo of a sound card audio interface"
]

JUNK_LABELS = [
    "a screenshot of a website or software interface with text and buttons",
    "a photo of a person's face or body",
    "a blurred, dark, or low quality image",
    "a photo of food on a plate or table",
    "a photo of animals or pets",
    "a photo of nature, landscapes, or outdoors",
    "a photo of a complete desk setup or room interior",
    "a digital illustration, logo, or graphic design",
    "a photo of a laptop or tablet device",
    "a photo of a smartphone or mobile device",
    "a diagram, chart, or technical drawing",
    "a photo of cables, peripherals, or accessories only"
]

In [4]:
def filter_images_for_hardvisionai(image_path, threshold=0.6):
    image = Image.open(image_path)
    
    all_labels = COMPONENT_LABELS + JUNK_LABELS
    
    inputs = processor(text=all_labels, images=image, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Probabilidades de todas las etiquetas
    probs = outputs.logits_per_image.softmax(dim=1)[0]
    
    # Probabilidad de componente
    prob_total_componente = sum(probs[:len(COMPONENT_LABELS)]).item()
    
    # Probabilidad de junk
    prob_total_junk = sum(probs[len(COMPONENT_LABELS):]).item()
    
    idx_max = torch.argmax(probs).item()
    winner = all_labels[idx_max]
    prob_winner = probs[idx_max].item()

    print(f"  Hardware total: {prob_total_componente:.2%} | Junk total: {prob_total_junk:.2%}")

    is_valid = (prob_total_componente > threshold 
                 and winner in COMPONENT_LABELS 
                 and prob_winner > 0.15)
    
    return is_valid

In [6]:
def pipeline_classifier(image_path):
    print(f"\n{'='*60}")
    if filter_images_for_hardvisionai(image_path, threshold=0.65):
        print("  ✅ Es un componente. Procesando con el modelo principal...")
        
        # TODO: modelo de Deep Learning de HardVisionAI
        # resultado = model.predict(ruta_imagen)
        # return resultado
        return "Clasificación exitosa"
    else:
        print("  ❌ Imagen rechazada: No parece ser un componente de PC.")
        return None



In [7]:
import os

# Probar todas las imágenes de img/tests
img_dir = '../img/tests'
img_files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

print('\n' + '='*60)
print('    PRUEBAS CON IMÁGENES DE img/tests')
print('='*60)

resultados_img = {'aceptadas': 0, 'rechazadas': 0}
for fname in img_files:
    print(f'\n📷 Imagen: {fname}')
    resultado = pipeline_classifier(os.path.join(img_dir, fname))
    if resultado:
        resultados_img['aceptadas'] += 1
    else:
        resultados_img['rechazadas'] += 1

print(f"\n{'='*60}")
print(f"RESUMEN img/tests: {resultados_img['aceptadas']} aceptadas, {resultados_img['rechazadas']} rechazadas")
print('='*60)


    PRUEBAS CON IMÁGENES DE img/tests

📷 Imagen: bale_lesion.jpg

  Hardware total: 2.30% | Junk total: 97.70%
  ❌ Imagen rechazada: No parece ser un componente de PC.

📷 Imagen: case_fan_8001.jpg

  Hardware total: 99.89% | Junk total: 0.11%
  ✅ Es un componente. Procesando con el modelo principal...

📷 Imagen: cpu.jpg

  Hardware total: 96.93% | Junk total: 3.07%
  ✅ Es un componente. Procesando con el modelo principal...

📷 Imagen: dataset_components_03.png

  Hardware total: 28.58% | Junk total: 71.42%
  ❌ Imagen rechazada: No parece ser un componente de PC.

📷 Imagen: ferrari_laferrari.jpg

  Hardware total: 8.37% | Junk total: 91.63%
  ❌ Imagen rechazada: No parece ser un componente de PC.

📷 Imagen: impact-of-outliers.png

  Hardware total: 2.46% | Junk total: 97.54%
  ❌ Imagen rechazada: No parece ser un componente de PC.

📷 Imagen: motherboard_713.jpg

  Hardware total: 94.28% | Junk total: 5.72%
  ✅ Es un componente. Procesando con el modelo principal...

📷 Imagen: pc_case_4